Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

This exercise covers two aspects:
* In tasks 1-6 you will implement mechanisms that allow training deeper models (better initialization, batch normalization). Note that for dropout and batch norm you are expected to implement it yourself without relying on ready-made components from Pytorch. After doing each of the tasks you can look at the plots and check how your changes impact gradients of network layers.
* In task 7 you will implement a convnet using [conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).


Tasks:
1. Check that the given implementation reaches 95% test accuracy for
   architecture input-64-64-10 in a few thousand batches.
2. Improve initialization and check that the network learns much faster
   and reaches over 97% test accuracy. A good basic initialization scheme is so-called Glorot initialization. For a set of weights going from a layer with $n_{in}$ neurons to a layer with $n_{out}$ neurons, it samples each weight from normal distribution with $0$ mean and standard deviation of $\sqrt{\frac{2}{n_{in}+n_{out}}}$.  
Check how better initialization changes distribution of gradients at the first epoch.
3. Check, that with proper initialization we can train architecture
   input-64-64-64-64-64-10, while with bad initialization it does
   not even get off the ground.
4. Add dropout implemented in pytorch (but without using torch.nn.Dropout)
5. Check that with 10 hidden layers (64 units each) even with proper
    initialization the network has a hard time to start learning.
6. Implement batch normalization (use train mode also for testing - it should perform well enough):
    * compute batch mean and variance
    * add new variables beta and gamma
    * check that the networks learns much faster for 5 layers
    * check that the network learns even for 10 hidden layers.
    * check how gradients change in comparison to network without batch norm.
7. So far we worked with a fully connected network. Design and implement in pytorch (by using pytorch functions) a simple convolutional network and achieve 99% test accuracy. The architecture is up to you, but even a few convolutional layers should be enough.

In [47]:
import sys
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()

In [48]:
#@title Visualize gradients

class GradientVisualizer:
    def __init__(self, net, num_epochs):
        self.num_epochs = num_epochs
        self.linear_layers = self.get_linear_layers(net)

        self.grad_to_weight_fig = None
        self.grads_in_layers_fig = None
        self.grads_at_epochs_fig = None
        self.init_figures()

    def get_linear_layers(self, net):
        linear_layers = []
        for field in net.__dict__['_modules'].values():
            if isinstance(field, Linear):
                linear_layers.append(field)
            if isinstance(field, nn.ModuleList):
                for module in field:
                    if isinstance(module, Linear):
                        linear_layers.append(module)

        assert linear_layers, \
        ('No linear layers found. Linear layers should be parameters of the network or they '
        'should be placed in a ModuleList which is a parameter of the network.')
        return linear_layers

    def get_epochs_for_one_layer(self):
        """
        We want to show gradient distributions from up to 7 selected epochs
        for one linear layer.
        """
        if self.num_epochs < 7:
            return list(range(self.num_epochs))
        else:
            return torch.linspace(0, self.num_epochs - 1, 7).int().tolist()

    def get_three_epochs(self):
        """
        We want to show gradients distributions from all layers at each of
        three epochs: first, middle and last.
        """
        return [0, self.num_epochs // 2, self.num_epochs - 1]

    def rgb_to_rgba(self, rgb_color, epoch):
        """
        Value of epoch parameter determines how transparent color should be
        in comparison to others.
        Colors for earlier epochs should be more transparent/less visible.
        """
        return f'rgba{rgb_color[3:-1]},{0.6 * (epoch + 1) / self.num_epochs + 0.15})'

    def init_figures(self):
        # Initialize figure with gradient to weight ratio plot
        fig = go.Figure()
        fig.update_layout(
            title='Gradient standard deviation to weight standard deviation ratio', title_x=0.5,
            xaxis_title='Epoch',
            yaxis_title='Gradient to weight ratio (log scale)',
            height=400, width=1500, margin=dict(b=10, t=60)
        )
        fig.update_yaxes(type='log')
        for i in range(len(self.linear_layers)):
            fig.add_trace(go.Scatter(
                x=[], y=[],
                mode='lines+markers', name=f'Linear layer {i}'
            ))

        self.grad_to_weight_fig = go.FigureWidget(fig)
        display(self.grad_to_weight_fig)

        # Initialize figure visualizing gradient distributions in layers
        num_rows = (len(self.linear_layers) - 1) // 3 + 1
        fig = make_subplots(
            rows=num_rows, cols=3,
            subplot_titles=[f'Linear layer {i}' for i in range(len(self.linear_layers))],
            vertical_spacing=0.2 / num_rows
        )
        fig.update_layout(
            title='Comparison between epochs of gradient distributions in layers', title_x=0.5,
            height=num_rows * 400, width=1500, margin=dict(b=10, t=60)
        )

        colors, _ = px.colors.convert_colors_to_same_type(2 * px.colors.qualitative.Plotly)
        for layer_num in range(len(self.linear_layers)):
            row = layer_num // 3 + 1
            col = layer_num % 3 + 1
            fig.update_xaxes(
                title_text='Gradient value', range=(-0.1, 0.1), row=row, col=col
            )
            fig.update_yaxes(
                title_text='Density (log scale)', type='log', row=row, col=col
            )

            # Create empty traces and update them later with actual gradient distributions.
            # Unfortunately, we cannot add new traces dynamically because Colab has problem
            # with widgets from plotly (traces added dynamically are rendered twice).
            for epoch in self.get_epochs_for_one_layer():
                fig.add_trace(
                    go.Scatter(
                        mode='lines', name=f'Epoch {epoch + 1}',
                        line=dict(color=self.rgb_to_rgba(colors[layer_num], epoch)),
                        legendgroup=layer_num
                    ),
                    row=row, col=col
                )

        self.grads_in_layers_fig = go.FigureWidget(fig)
        display(self.grads_in_layers_fig)

        # Initialize figure comparing gradient distributions between layers at the
        # first, middle and last epoch
        selected_epochs_indices = self.get_three_epochs()
        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=[f'Epoch {epoch + 1}' for epoch in selected_epochs_indices]
        )
        fig.update_layout(
            title='Comparison between layers of gradient distributions at epochs', title_x=0.5,
            height=400, width=1500, margin=dict(b=10, t=60)
        )

        for col, epoch in enumerate(selected_epochs_indices, 1):
            fig.update_yaxes(title_text='Density (log scale)', type='log', row=1, col=col)
            fig.update_xaxes(
                title_text='Gradient value',
                range=(-0.05, 0.05) if epoch != 0 else (-1, 1),
                row=1, col=col
            )

            # Create empty traces and update them later with actual gradient distributions.
            for layer_num in range(len(self.linear_layers)):
                fig.append_trace(
                    go.Scatter(
                        mode='lines', name=f'Linear layer {layer_num}',
                        line=dict(color=colors[layer_num]), showlegend=(col == 1)
                    ),
                    row=1, col=col
                )

        self.grads_at_epochs_fig = go.FigureWidget(fig)
        display(self.grads_at_epochs_fig)

    def visualize_gradients(self, lr, epoch, batch_idx):
        # It is enough to use gradients calculated for the first batch.
        if batch_idx != 0:
            return

        epoch_grads = []
        epoch_grad_to_weight_ratios = []
        for layer in self.linear_layers:
            epoch_grads.append(layer.weight.grad.flatten().detach())
            epoch_grad_to_weight_ratios.append(
                (lr * layer.weight.grad.std() / layer.weight.std()).item()
            )

        # Update figure with gradient to weight ratio plot
        for i, grad_to_weight_ratio in enumerate(epoch_grad_to_weight_ratios):
            x = self.grad_to_weight_fig.data[i].x
            next_x_val = x[-1] + 1 if x else 1
            self.grad_to_weight_fig.data[i].x += (next_x_val, )
            self.grad_to_weight_fig.data[i].y += (grad_to_weight_ratio, )

        # Update figure visualizing gradient distributions in layers
        selected_epochs = self.get_epochs_for_one_layer()
        if epoch in selected_epochs:
            epoch_idx = selected_epochs.index(epoch)
            for layer_num, layer_grad in enumerate(epoch_grads):
                trace_idx = layer_num * len(selected_epochs) + epoch_idx
                hy, hx = torch.histogram(layer_grad, bins=50, density=True)
                hy = hy / max(hy) + 0.001
                self.grads_in_layers_fig.data[trace_idx].x = hx[:-1].tolist()
                self.grads_in_layers_fig.data[trace_idx].y = hy.tolist()

        # Update figure visualizing gradient distributions at epochs
        selected_epochs = self.get_three_epochs()
        if epoch in selected_epochs:
            epoch_idx = selected_epochs.index(epoch)
            for layer_num, layer_grad in enumerate(epoch_grads):
                trace_idx = epoch_idx * len(self.linear_layers) + layer_num
                hy, hx = torch.histogram(layer_grad, bins=50, density=True)
                hy = hy / max(hy) + 0.001
                self.grads_at_epochs_fig.data[trace_idx].x = hx[:-1].tolist()
                self.grads_at_epochs_fig.data[trace_idx].y = hy.tolist()

In [49]:
class Linear(torch.nn.Module):
    def __init__(self, in_features, out_features, glorot=False, batch_norm=False, eps=1e-6):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.gamma = Parameter(torch.Tensor(in_features))
        self.beta = Parameter(torch.Tensor(in_features))
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.eps = eps
        self.glorot = glorot
        self.bn = batch_norm
        self.reset_glorot()

    def reset_glorot(self):
        if self.glorot:
            stddev = (2 / (self.in_features + self.out_features)) ** (1/2)
        else:
            stddev = 0.25

        self.weight.data.normal_(mean = 0, std = stddev)
        init.zeros_(self.beta)
        init.zeros_(self.bias)
        init.ones_(self.gamma)

    def forward(self, x):
        if self.bn:
            batch_size, _ = x.shape 
        
            means = (1 / batch_size) * torch.sum(x, dim = 0)
            means = means.repeat(batch_size, 1)
            variances = (1 / batch_size) * torch.sum((x - means)**2, dim = 0)
            variances = variances.repeat(batch_size, 1)

            numerator = x - means
            denominator = (variances + self.eps)**(1/2)
            x = self.gamma * (numerator / denominator) + self.beta

            return x.matmul(self.weight.t())
        else:
            return x.matmul(self.weight.t()) + self.bias

In [50]:
class Dropout(torch.nn.Module):
    def __init__(self, features, p=0.95):
        super(Dropout, self).__init__()
        self.features = features
        self.p = p

    def forward(self, x):
        dropout = torch.Tensor(x.shape)
        dropout.bernoulli_(p = self.p)
        new_x = x * dropout

        return new_x * (1/self.p)
    
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x 

In [51]:
from functools import reduce
import numpy as np

class Net(nn.Module):
    def __init__(self, sizes, glorot=False, dropout=False, batch_norm=False):
        super(Net, self).__init__()

        dropouts = [Dropout(s) for s in sizes[1:-1]] + [Identity()]
        sizes = np.array(sizes)
        sizes2 = np.vstack([sizes[:-1], sizes[1:]]).T
        modules = [Linear(*s, glorot=glorot, batch_norm=batch_norm) for s in sizes2]

        self.fcs = nn.ModuleList(modules)
        self.dps = nn.ModuleList(dropouts)
        self.dropout = dropout

    def forward(self, x):
        activasion_funs = [F.relu] * (len(self.fcs) - 1) + [lambda x : x]

        x = x.view(-1, 28 * 28)

        tuple_list = zip(self.fcs, self.dps, activasion_funs)

        for fc, dp, activation_fun in tuple_list:
            x = activation_fun(fc(x))

            if self.dropout:
                x = dp(x)

        return x

In [52]:
class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet, self).__init__()
        self.fc1 = Linear(784, 64)
        self.dp1 = Dropout(64)
        self.fc2 = Linear(64, 64)
        self.dp2 = Dropout(64)
        self.fc3 = Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = F.relu(self.fc2(x))
        x = self.dp2(x)
        x = self.fc3(x)
        return x

class BatchNormNet(nn.Module):
    def __init__(self, dropout=False):
        super(BatchNormNet, self).__init__()
        self.fc1 = Linear(784, 64, True)
        self.dp1 = Dropout(64) if dropout else Identity()
        self.fc2 = Linear(64, 64, True)
        self.dp2 = Dropout(64) if dropout else Identity()
        self.fc3 = Linear(64, 10, True)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = F.relu(self.fc2(x))
        x = self.dp2(x)
        x = self.fc3(x)
        return x

In [53]:
class MnistTrainer(object):
    def __init__(self, batch_size):
        transform = transforms.Compose(
                [transforms.ToTensor()])
        self.trainset = torchvision.datasets.MNIST(
            root='./data',
            download=True,
            train=True,
            transform=transform)
        self.trainloader = torch.utils.data.DataLoader(
            self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

        self.testset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(
            self.testset, batch_size=batch_size, shuffle=False, num_workers=2)

    def train(self, net, gradient_visualizer, epochs=20, lr=0.05, momentum=0.9):
        gv = gradient_visualizer is not None

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

        for epoch in range(epochs):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader):
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                if gv:
                    gradient_visualizer.visualize_gradients(lr, epoch, i)
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} test images: {} %'.format(
                total, 100 * correct / total))

In [54]:
%%capture
epochs = 10

# no glorot - crashes on 7
# with glorot - sometimes crashes on ~15
# glorot + batchnorm - easily trains on 19
# cannot test highet bcs of some bug in gradient visualizer ;(
hidden_layers = 19
net = Net([784] + [64] * hidden_layers + [10], glorot = True, dropout = False, batch_norm = True)
gradient_visualizer = GradientVisualizer(net, epochs)

In [56]:
trainer = MnistTrainer(batch_size=128)
trainer.train(net, gradient_visualizer, epochs=epochs)

[1,   100] loss: 0.235
[1,   200] loss: 0.228
[1,   300] loss: 0.235
[1,   400] loss: 0.233
Accuracy of the network on the 10000 test images: 92.85 %
[2,   100] loss: 0.210
[2,   200] loss: 0.219
[2,   300] loss: 0.214
[2,   400] loss: 0.216
Accuracy of the network on the 10000 test images: 93.51 %
[3,   100] loss: 0.185
[3,   200] loss: 0.190
[3,   300] loss: 0.195
[3,   400] loss: 0.194
Accuracy of the network on the 10000 test images: 93.61 %
[4,   100] loss: 0.184
[4,   200] loss: 0.177
[4,   300] loss: 0.180
[4,   400] loss: 0.180
Accuracy of the network on the 10000 test images: 94.14 %
[5,   100] loss: 0.158
[5,   200] loss: 0.157
[5,   300] loss: 0.175
[5,   400] loss: 0.163
Accuracy of the network on the 10000 test images: 94.01 %
[6,   100] loss: 0.155
[6,   200] loss: 0.153
[6,   300] loss: 0.136
[6,   400] loss: 0.160
Accuracy of the network on the 10000 test images: 94.23 %
[7,   100] loss: 0.141
[7,   200] loss: 0.138
[7,   300] loss: 0.140
[7,   400] loss: 0.138
Accuracy

In [45]:
class ConvolutionalNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

In [ ]:
epochs = 10

net = ConvolutionalNet()
trainer = MnistTrainer(batch_size=128)
trainer.train(net, None, epochs=epochs)

[1,   100] loss: 1.609
[1,   200] loss: 0.212
[1,   300] loss: 0.132
[1,   400] loss: 0.109
Accuracy of the network on the 10000 test images: 97.2 %
[2,   100] loss: 0.088
[2,   200] loss: 0.077
[2,   300] loss: 0.074
[2,   400] loss: 0.076
Accuracy of the network on the 10000 test images: 98.12 %
[3,   100] loss: 0.054
[3,   200] loss: 0.052
[3,   300] loss: 0.048
[3,   400] loss: 0.057
Accuracy of the network on the 10000 test images: 97.96 %
[4,   100] loss: 0.046
[4,   200] loss: 0.048
[4,   300] loss: 0.042
[4,   400] loss: 0.041
Accuracy of the network on the 10000 test images: 98.38 %
[5,   100] loss: 0.036
[5,   200] loss: 0.032
[5,   300] loss: 0.040
[5,   400] loss: 0.038
Accuracy of the network on the 10000 test images: 98.14 %
[6,   100] loss: 0.030
[6,   200] loss: 0.027
[6,   300] loss: 0.037
[6,   400] loss: 0.040
Accuracy of the network on the 10000 test images: 98.52 %
[7,   100] loss: 0.027
[7,   200] loss: 0.028
[7,   300] loss: 0.029
[7,   400] loss: 0.029
Accuracy 